In [4]:
import pandas as pd
import os
import psycopg2
from psycopg2 import extras


csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('payments.csv', 'payments'),
    ('geolocation.csv', 'geolocation'),
    ('order_items.csv', 'order_items')
]

# Connect to PostgreSQL database
conn = psycopg2.connect(
    host='localhost',
    user='postgres',
    password='uweria123',
    dbname='ecomarce', 
    port = "5433"
)
cursor = conn.cursor()

folder_path = r'E:\\SQLPostgrast project'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'REAL'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    df = df.where(pd.notnull(df), None)
    
    print(f"Processing {csv_file}...")
    
    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]
    
    # Create table
    columns_def = ', '.join([f'"{col}" {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_query = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({columns_def});'
    cursor.execute(create_query)
    
    # Bulk insert using execute_values
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ', '.join([f'"{col}"' for col in df.columns])
    query = f'INSERT INTO "{table_name}" ({cols}) VALUES %s'
    extras.execute_values(cursor, query, tuples)
    
    conn.commit()

conn.close()
print("All CSV files imported into PostgreSQL successfully !")


Processing customers.csv...
Processing orders.csv...
Processing sellers.csv...
Processing products.csv...
Processing payments.csv...
Processing geolocation.csv...
Processing order_items.csv...
All CSV files imported into PostgreSQL successfully !
